In [31]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import requests

In [32]:
# EXTRACT ALL SONG TITLES ALONG WITH THEIR SINGERS/COMPOSERS VIA BEAUTIFULSOUP
# https://open.spotify.com/playlist/69HIaVaMTLoMYpUoQkgYdv?si=7a61a1d6ab994afd

def get_all_songs(playlist_url):
    
    # Manually create Chrome options
    chrome_options = webdriver.ChromeOptions()
    # Add any additional options if needed

    # Manually create the Chrome service
    chrome_service = webdriver.chrome.service.Service(ChromeDriverManager().install())

    # Pass options and service to Chrome constructor
    driver = webdriver.Chrome(options=chrome_options, service=chrome_service)

    driver.get(playlist_url)

    driver.implicitly_wait(10)
    xpath_expression = "//div[contains(@class, 'Text__TextElement-sc-if376j-0') and contains(@class, 'ksSRyh') and contains(@class, 'encore-text-body-medium') and contains(@class, 't_yrXoUO3qGsJS4Y6iXX') and contains(@class, 'standalone-ellipsis-one-line')]"
#     element = driver.find_element(By.XPATH, xpath_expression)
    wait = WebDriverWait(driver,1000)
    wait.until(EC.presence_of_element_located((By.XPATH,xpath_expression)))

    # Get the updated page source with dynamic content
    html_content = driver.page_source

    driver.quit()

    data = BeautifulSoup(html_content,'html.parser')
#     print(data)
    song_names = data.find_all('div', class_='Text__TextElement-sc-if376j-0 ksSRyh encore-text-body-medium t_yrXoUO3qGsJS4Y6iXX standalone-ellipsis-one-line')
    composers = []
    composers = data.find_all('div',class_='Text__TextElement-sc-if376j-0 gYdBJW encore-text-body-small')
    if (len(composers) == 0):
        composers = data.find_all('span',class_='Text__TextElement-sc-if376j-0 gYdBJW encore-text-body-small')
    search_input = []
    print(song_names)
    print(composers)
    for i in range(len(song_names)):
        temp = song_names[i].text
        temp+=" - "
    #     print(temp)
        temp_composers = composers[i].find_all('a')
        for j in temp_composers:
            temp+= j.text+", "
    #     print(temp)
        search_input.append(temp)
    return search_input

In [33]:
# SEARCH FOR EACH SONG IN YOUTUBE AND GET THERE URLS

def get_all_urls(search_input):

    # Manually create Chrome options
    chrome_options = webdriver.ChromeOptions()
    # Add any additional options if needed

    # Manually create the Chrome service
    chrome_service = webdriver.chrome.service.Service(ChromeDriverManager().install())

    # Pass options and service to Chrome constructor
    driver = webdriver.Chrome(options=chrome_options, service=chrome_service)

    driver.get('https://www.youtube.com/')

    wait = WebDriverWait(driver,100000)
    wait.until(EC.presence_of_element_located((By.XPATH,'//input[@name = "search_query"]')))
    search_box = driver.find_element(By.XPATH,'//input[@name = "search_query"]')
    yt_urls = []
    for i in search_input:
    #   IT WILL SEARCH FOR THE YOUTUBE URL OF THAT SONG
        search_box = driver.find_element(By.XPATH,'//input[@name = "search_query"]')
        search_box.clear()
        search_box.send_keys(i)
        search_box.submit()
    #   NOW WE WILL APPLY A WAIT TO GET THE YOUTUBE LINK
        wait = WebDriverWait(driver,100000)
        wait.until(EC.presence_of_element_located((By.ID,'video-title')))
        yt_link = driver.find_element(By.ID,'video-title')
        yt_link.click()
        wait = WebDriverWait(driver,100000)
        wait.until(EC.presence_of_element_located((By.XPATH,'//button[contains(@class,"ytp-mute-button ytp-button")]')))
    #     mute_button = driver.find_element(By.XPATH,'//button[contains(@class,"ytp-mute-button ytp-button")]')
    #     mute_button.click()
        yt_urls.append(driver.current_url)
        driver.get('https://www.youtube.com/')
    return yt_urls

In [34]:
# DOWNLOADING SONGS

def download_songs(yt_urls):

    chrome_options = webdriver.ChromeOptions()
    # Add any additional options if needed

    # Manually create the Chrome service
    chrome_service = webdriver.chrome.service.Service(ChromeDriverManager().install())

    # Pass options and service to Chrome constructor
    driver = webdriver.Chrome(options=chrome_options, service=chrome_service)
    driver.get('https://ytmp3.nu/Q2rp/')
    count = 0
    for i in yt_urls:
        wait = WebDriverWait(driver,1000)
        wait.until(EC.presence_of_element_located((By.NAME,'url')))
        search_box = driver.find_element(By.NAME,'url')
        search_box.clear()
    #     print(search_box.get_attribute('outerHTML'))
        search_box.send_keys(i)
        search_box.submit()
        wait = WebDriverWait(driver,1000)
        wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT,'Convert next')))
        try:
            download_button = driver.find_element(By.LINK_TEXT,'Download')
            download_button.click()
            driver.implicitly_wait(10)
        except NoSuchElementException:
            index = yt_urls.index(i)
            print(search_input[index] + " - NOT DOWNLOADABLE")
    #     print(download_button.get_attribute('outerHTML'))
        convert_next_button = driver.find_element(By.LINK_TEXT,'Convert next')
        convert_next_button.click()
    driver.implicitly_wait(1000)

In [36]:
# SAMPLE INPUT - https://open.spotify.com/playlist/69HIaVaMTLoMYpUoQkgYdv?si=7a61a1d6ab994afd
print("--------WELCOME TO PLAYLIST-DOWNLOAD-CENTRE--------")
playlist_url = input("ENTER PLAYLIST URL : ")
search_input = get_all_songs(playlist_url)
print("GOT SEARCH INPUT")
yt_urls = get_all_urls(search_input)
print("GOT ALL URLS")
download_songs(yt_urls)
print("DOWNLOADED SUCCESSFULLY")
